# MetaDSL
**Implicit types and why we need something more structured**

The goal of this notebook is to demonstrate the initial support for creating expressions of different types.

Also to show why defining types implicitly has some drawbacks and next steps

In [17]:
from metadsl import *

import dataclasses

First, let's make a natural number type, and define some operations on it:

In [24]:
@dataclasses.dataclass(frozen=True)
class NaturalNumber(Instance):

    def __add__(self, other: "NaturalNumber") -> "NaturalNumber":
        return NaturalNumber(Call(NaturalNumber.__add__, (self, other)))
    
    def __mul__(self, other: "NaturalNumber") -> "NaturalNumber":
        return NaturalNumber(Call(NaturalNumber.__mul__, (self, other)))

Now we can create some natural numbers and execute some operations on them:

In [25]:
n = (NaturalNumber(1) + NaturalNumber(2)) * NaturalNumber(3)
n

NaturalNumber(__value__=Call(function=<function NaturalNumber.__mul__ at 0x106f852f0>, args=(NaturalNumber(__value__=Call(function=<function NaturalNumber.__add__ at 0x106f85268>, args=(NaturalNumber(__value__=1), NaturalNumber(__value__=2)))), NaturalNumber(__value__=3))))

Now let's define a way to execute these natural numbers:

In [38]:
def execute_call_fn(call):
    if call.function == NaturalNumber.__add__:
        return call.args[0].value + call.args[1].value
    elif call.function == NaturalNumber.__mul__:
        return call.args[0].value * call.args[1].value
    raise NotImplementedError()

identity = lambda v: v
execute_expr = lambda expr: expr.fold(expr_fn=identity, call_value_fn=execute_call_fn, other_value_fn=identity)
execute_natural = lambda n: execute_expr(Expression.from_instance(n)).instance

In [39]:
execute_natural(n)

NaturalNumber(__value__=9)